In [1]:
import pandas as pd
from tqdm import tqdm
import re
import os
from util import  *
from rdkit import Chem
tqdm.pandas()
import json
from common import *

In [2]:
#input
YMDB_total_smiles_path = '../../Data/database/ymdb/ymdb_final_result.xlsx'
#output

In [3]:
YMDB_Data = pd.read_excel(YMDB_total_smiles_path)
YMDB_Data.head()

,ID,inchikey,SMILES,NAME,super_class,class,standard_smiles,inchikey0,in_model,has_carbon
0,YMDB00001,VDXLUNDMVKSKHO-XVFCMESISA-N,[H]O[C@@]1([H])[C@@]([H])(O[C@]([H])(C([H])([H...,5'-phosphoribosyl-n-formylglycinamide,"Nucleosides, nucleotides, and analogues",Glycinamide ribonucleotides,O=CNCC(=O)N[C@@H]1O[C@H](COP(=O)(O)O)[C@@H](O)...,VDXLUNDMVKSKHO,1,1.0
1,YMDB00002,ZDXPYRJPNDTMRX-VKHMYHEASA-N,[H]OC(=O)[C@@]([H])(N([H])[H])C([H])([H])C([H]...,l-glutamine,Organic acids and derivatives,Carboxylic acids and derivatives,NC(=O)CC[C@H](N)C(=O)O,ZDXPYRJPNDTMRX,1,1.0
2,YMDB00003,XSQUKJJJFZCRTK-UHFFFAOYSA-N,[H]N([H])C(=O)N([H])[H],urea,Organic acids and derivatives,Organic carbonic acids and derivatives,NC(N)=O,XSQUKJJJFZCRTK,1,1.0
3,YMDB00004,HXXFSFRBOHSIMQ-FPRJBGLDSA-N,[H]OC([H])([H])[C@@]1([H])O[C@]([H])(OP(=O)(O[...,galactose 1-phosphate,Organic oxygen compounds,Organooxygen compounds,O=P(O)(O)O[C@H]1O[C@H](CO)[C@H](O)[C@H](O)[C@H]1O,HXXFSFRBOHSIMQ,1,1.0
4,YMDB00005,QAOWNCQODCNURD-UHFFFAOYSA-L,[O-]S([O-])(=O)=O,sulfate,Homogeneous non-metal compounds,Non-metal oxoanionic compounds,O=S(=O)([O-])[O-],QAOWNCQODCNURD,1,0.0


In [4]:
yeast8U_del2_path = '../../Data/model/Yeast-MetaTwin.yml'
yeast8U_mod = cobra.io.load_yaml_model(yeast8U_del2_path)

In [5]:
rxndb_to_model_with_target_path = '../../Results/not_lipid/top50_0.3_add_no_ec_re/rxndb_to_model_with_target_top50_0.3.csv'
rxndb_to_model_add_no_ec_with_target = pd.read_csv(rxndb_to_model_with_target_path)

In [6]:
rxndb_to_model_add_no_ec_with_target = rxndb_to_model_add_no_ec_with_target[rxndb_to_model_add_no_ec_with_target['EC number'].isna()]
print(rxndb_to_model_add_no_ec_with_target.shape)
rxndb_to_model_add_no_ec_with_target.head(1)

(40, 5)


,NO,EC number,templateID,rxn_smiles_basic,rxn_smiles_final
3,rxn1076,NaN,MNXR164692,C[S@@+](CCC([NH3+])C(=O)[O-])CC1OC(n2cnc3c(N)n...,C[S@@+](CCC([NH3+])C(=O)[O-])CC1OC(n2cnc3c(N)n...


In [7]:
YMDB_id = 'YMDB16061'
for i in yeast8U_mod.metabolites:
    if YMDB_id in i.name:
        print(i.id,i.name)

sn_257 YMDB16061


In [8]:
target_smiles_list = ['O=P(O)(O)OCC1OC(O)(CO)C(O)C(O)C1O',
 'CC1=C(O)C(=O)OC1C',
 'O=C(C=Cc1ccc(O)c(O)c1)OC(C(=O)O)C(O)C(=O)O',
 'COc1cc(C=CC(=O)OC(C(=O)O)C(O)C(=O)O)ccc1O',
 'O=C(O)C(CO)c1ccccc1',
 'Cc1ccc(O)c([N+](=O)[O-])c1']

In [9]:
def metid_inGEM(YMDB_id,yeast8U_mod):
    met_id_list = []
    for i in yeast8U_mod.metabolites:
        if YMDB_id in i.name:
            met_id_list.append(i.id)
    return met_id_list

In [10]:
for target_smiles in target_smiles_list:
    if smiles_has_carbon(target_smiles):
        for index,row in YMDB_Data.iterrows():
            if compare_smiles_inchikey(row['SMILES'],target_smiles):
                print(target_smiles,row['ID'])

O=P(O)(O)OCC1OC(O)(CO)C(O)C(O)C1O YMDB00196
CC1=C(O)C(=O)OC1C YMDB01572
O=C(C=Cc1ccc(O)c(O)c1)OC(C(=O)O)C(O)C(=O)O YMDB01646
COc1cc(C=CC(=O)OC(C(=O)O)C(O)C(=O)O)ccc1O YMDB01693
O=C(O)C(CO)c1ccccc1 YMDB02309
Cc1ccc(O)c([N+](=O)[O-])c1 YMDB15936


In [ ]:
# YMDB02309
# YMDB15936
# YMDB00196
# YMDB01572
# O=C(C=Cc1ccc(O)c(O)c1)OC(C(=O)O)C(O)C(=O)O YMDB01646
# COc1cc(C=CC(=O)OC(C(=O)O)C(O)C(=O)O)ccc1O YMDB01693

target_smiles_list = ['O=P(O)(O)OCC1OC(O)(CO)C(O)C(O)C1O',
 'CC1=C(O)C(=O)OC1C',
#  'O=C(C=Cc1ccc(O)c(O)c1)OC(C(=O)O)C(O)C(=O)O',
#  'COc1cc(C=CC(=O)OC(C(=O)O)C(O)C(=O)O)ccc1O',
 'O=C(O)C(CO)c1ccccc1',
 'Cc1ccc(O)c([N+](=O)[O-])c1']

In [11]:
for target_smiles in target_smiles_list:
    print(target_smiles)
    for index1,row1 in rxndb_to_model_add_no_ec_with_target.iterrows():
        smiles_list = row1['rxn_smiles_basic'].replace('>>','.').split('.')
        if any(compare_smiles_inchikey(target_smiles,x) for x in smiles_list):
            print('==',row1['NO'],target_smiles,row1['rxn_smiles_basic'])
            for smiles in smiles_list:
                if smiles_has_carbon(smiles):
                    for index2,row2 in YMDB_Data.iterrows():
                        if compare_smiles_inchikey(row2['SMILES'],smiles):
                            print('====',smiles,row2['ID'],metid_inGEM(row2['ID'],yeast8U_mod))
    print('==============================================')

O=P(O)(O)OCC1OC(O)(CO)C(O)C(O)C1O
== rxn77645 O=P(O)(O)OCC1OC(O)(CO)C(O)C(O)C1O O=C(CO)C(O)C(O)C(O)C(O)COP(=O)(O)O>>O=P(O)(O)OCC1OC(O)(CO)C(O)C(O)C1O
==== O=C(CO)C(O)C(O)C(O)C(O)COP(=O)(O)O YMDB00752 []
==== O=P(O)(O)OCC1OC(O)(CO)C(O)C(O)C1O YMDB00196 ['sn_39']
== rxn47961 O=P(O)(O)OCC1OC(O)(CO)C(O)C(O)C1O O=P([O-])([O-])OCC1OC(O)(CO)C(O)C(O)C1O>>O=C(CO)C(O)C(O)C(O)C(O)COP(=O)([O-])[O-]
==== O=P([O-])([O-])OCC1OC(O)(CO)C(O)C(O)C1O YMDB00196 ['sn_39']
==== O=C(CO)C(O)C(O)C(O)C(O)COP(=O)([O-])[O-] YMDB00752 []
== rxn47962 O=P(O)(O)OCC1OC(O)(CO)C(O)C(O)C1O O=P(O)(O)OCC1OC(O)(CO)C(O)C(O)C1O>>O=C(CO)C(O)C(O)C(O)C(O)COP(=O)(O)O
==== O=P(O)(O)OCC1OC(O)(CO)C(O)C(O)C1O YMDB00196 ['sn_39']
==== O=C(CO)C(O)C(O)C(O)C(O)COP(=O)(O)O YMDB00752 []
CC1=C(O)C(=O)OC1C
== rxn120668 CC1=C(O)C(=O)OC1C CC=O.CCC(=O)C(=O)[O-]>>CC1=C(O)C(=O)OC1C.O
==== CC=O YMDB00022 []
==== CCC(=O)C(=O)[O-] YMDB00071 []
==== CC1=C(O)C(=O)OC1C YMDB01572 ['sn_86']
== rxn10688 CC1=C(O)C(=O)OC1C CC1=C(O)C(=O)OC1C.O>>CCC(=O)C(=O)[O